In [11]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import csv
import time
import json
from pandas import json_normalize

In [12]:
df1 = pd.read_csv("10분 지하철 결측치.csv", encoding='cp949')
display(df1)

,Unnamed: 0,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,94,C1312,518,아파트,충청남도,국민임대,39.72,60,12,A,17460000.0,122210.0,NaN,3.0,527,359
1,95,C1312,518,아파트,충청남도,국민임대,39.98,89,12,A,17460000.0,122210.0,NaN,3.0,527,359
2,96,C1312,518,아파트,충청남도,국민임대,41.55,225,12,A,19954000.0,130940.0,NaN,3.0,527,359
3,97,C1312,518,아파트,충청남도,국민임대,46.90,143,12,A,28687000.0,149660.0,NaN,3.0,527,359
4,98,C1874,619,아파트,충청남도,영구임대,26.37,294,2,C,3141000.0,69900.0,NaN,2.0,97,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,2339,C1350,1401,아파트,대전광역시,공공임대(10년),59.91,13,2,A,37474000.0,519350.0,NaN,6.0,1636,2315
80,2340,C1350,1401,아파트,대전광역시,공공임대(10년),59.92,223,2,A,37385000.0,516130.0,NaN,6.0,1636,2315
81,2341,C1350,1401,아파트,대전광역시,공공임대(분납),51.99,146,2,A,54612000.0,131130.0,NaN,6.0,1636,2315
82,2342,C1350,1401,아파트,대전광역시,공공임대(분납),59.91,32,2,A,63585000.0,152680.0,NaN,6.0,1636,2315


In [13]:
df1.shape, df1.drop_duplicates().shape # 중복 존재

((84, 16), (84, 16))

In [17]:
대전 = pd.read_csv("공공임대추출데이터/공공임대주택단지_대전_아파트.csv", encoding='cp949')
충남 = pd.read_csv("공공임대추출데이터/공공임대주택단지_충남_아파트.csv", encoding='cp949')

임대 = pd.read_csv("임대주택단지_전처리.csv", encoding='cp949')

대전 = 대전.rename(columns={'공급전용면적':'전용면적', '광역시도 명':'지역'})
충남 = 충남.rename(columns={'공급전용면적':'전용면적', '광역시도 명':'지역'})

In [18]:
대전지역 = df1['지역'] == '대전광역시'
df1대전 = df1[대전지역]

충남지역 = df1['지역'] == '충청남도'
df1충남 = df1[충남지역]


display(df1대전)
display(df1충남)



merge_df1대전 = pd.merge(df1대전, 대전, on='전용면적')
merge_df1대전.to_csv('지역별합친데이터/트레인_대전_지하철_merge_cp949.csv', encoding='cp949')

merge_df1충남 = pd.merge(df1충남, 충남, on='전용면적')
merge_df1충남.to_csv('지역별합친데이터/트레인_충남_지하철_merge_cp949.csv', encoding='cp949')

merge_df1대전 = pd.merge(df1대전, 임대, on='전용면적')
merge_df1대전.to_csv('지역별합친데이터/임대_대전_지하철_merge_cp949.csv', encoding='cp949')

merge_df1충남 = pd.merge(df1충남, 임대, on='전용면적')
merge_df1충남.to_csv('지역별합친데이터/임대_충남_지하철_merge_cp949.csv', encoding='cp949')

,Unnamed: 0,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
7,144,C1424,625,아파트,대전광역시,공공임대(50년),39.99,268,2,A,14247000.0,198980.0,NaN,7.0,517,179
8,145,C1424,625,아파트,대전광역시,공공임대(50년),49.95,119,2,A,20173000.0,225100.0,NaN,7.0,517,179
9,146,C1424,625,아파트,대전광역시,공공임대(50년),49.95,237,2,A,21203000.0,225100.0,NaN,7.0,517,179
10,147,C2100,880,아파트,대전광역시,국민임대,51.51,46,4,A,22307000.0,89210.0,NaN,5.0,736,671
11,148,C2100,880,아파트,대전광역시,국민임대,51.59,35,4,A,22307000.0,89210.0,NaN,5.0,736,671
12,149,C2100,880,아파트,대전광역시,국민임대,51.90,589,4,A,22307000.0,89210.0,NaN,5.0,736,671
13,150,C2100,880,아파트,대전광역시,국민임대,59.91,118,4,A,27885000.0,111520.0,NaN,5.0,736,671
14,151,C2100,880,아파트,대전광역시,국민임대,59.94,46,4,A,27885000.0,111520.0,NaN,5.0,736,671
15,152,C2100,880,아파트,대전광역시,국민임대,59.99,46,4,A,27885000.0,111520.0,NaN,5.0,736,671
16,171,C2520,970,아파트,대전광역시,공공임대(50년),39.69,960,2,A,13155000.0,177500.0,NaN,2.0,420,386


,Unnamed: 0,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,94,C1312,518,아파트,충청남도,국민임대,39.72,60,12,A,17460000.0,122210.0,NaN,3.0,527,359
1,95,C1312,518,아파트,충청남도,국민임대,39.98,89,12,A,17460000.0,122210.0,NaN,3.0,527,359
2,96,C1312,518,아파트,충청남도,국민임대,41.55,225,12,A,19954000.0,130940.0,NaN,3.0,527,359
3,97,C1312,518,아파트,충청남도,국민임대,46.90,143,12,A,28687000.0,149660.0,NaN,3.0,527,359
4,98,C1874,619,아파트,충청남도,영구임대,26.37,294,2,C,3141000.0,69900.0,NaN,2.0,97,62
5,99,C1874,619,아파트,충청남도,영구임대,26.37,149,2,C,3141000.0,69900.0,NaN,2.0,97,62
6,100,C1874,619,아파트,충청남도,영구임대,31.32,149,2,C,3731000.0,83020.0,NaN,2.0,97,62
25,347,C1068,806,아파트,충청남도,국민임대,36.65,200,11,A,11234000.0,106090.0,NaN,2.0,804,510
26,348,C1068,806,아파트,충청남도,국민임대,36.98,130,11,A,11234000.0,106090.0,NaN,2.0,804,510
27,349,C1068,806,아파트,충청남도,국민임대,41.55,252,11,A,14981000.0,124830.0,NaN,2.0,804,510
